In [ ]:
#Start Here for Making Beam-Date Graphs
#format beamsize number
def format_number(number):
        # Case 1: If the number is less than 1 (e.g., 0.004567)
        if number < 0:
                # Format the positive part of the number and prepend the negative sign
                return "-" + format_number(abs(number))
        if number < 1:
                # Convert the number to a string with high precision
                num_str = f"{number:.16g}"

                # Identify the leading zeros and decimal point
                leading_part = []
                for char in num_str:
                        if char == '0' or char == '.':
                                leading_part.append(char)
                        else:
                                break

                # Remove leading zeros and the decimal point for significant digits
                significant_digits = "".join(char for char in num_str if char.isdigit() and char != "0")

                # Extract the first two non-zero digits
                if len(significant_digits) >= 2:
                        first_two = significant_digits[:2]
                elif len(significant_digits) == 1:
                        first_two = significant_digits + "0"  # Pad with zero if only one significant digit exists
                else:
                        first_two = "00"  # Handle edge case like 0

                # Combine the leading zeros and the first two non-zero digits
                return "".join(leading_part) + first_two

        # Case 2: If the number is greater than or equal to 1
        else:
                # Format to two decimal places
                return f"{number:.2f}"
        

In [ ]:
#Create Spectral Plots
import matplotlib.pyplot as pylab
import matplotlib.colors
import numpy as np
import shutil
import os
import matplotlib.cm as cmx
import math
from matplotlib.ticker import MultipleLocator
import copy
from matplotlib.colors import LinearSegmentedColormap
jet = pylab.get_cmap('jet')
colors = jet(np.linspace(0, 1, 256))
colors = colors[np.logical_not((colors[:,0] > 0.9) & (colors[:,1] > 0.9))]  # crude yellow filter
cm2 = LinearSegmentedColormap.from_list('jet_no_yellow', colors)

os.chdir('analysis/multi_freq_from_archive')

sourcesofinterest=('4258','1194','3079','4945','Circinus','1068','2273','4388','2960','3789')
#sourcesofinterest=('1194','4258')


def round_down_to_half_integer(num):
    return math.floor(num * 2) / 2

def round_up_to_half_integer(num):
    return math.ceil(num * 2) / 2

msize=10

path = f'SpectraGraphs'
if os.path.exists(path):
    shutil.rmtree(path)
os.mkdir(path)

source_text='fitsumfiles/final/fitsummary_final_withextras.txt'
freqs=[]
fluxs=[]
beams=[]

with open(source_text, 'r') as infile:
    jfin=-1
    for line in infile:
        if len(line)<2:
            continue
        line=eval(line)
        source=line[0]
        for i in sourcesofinterest:
            if i.lower() in source.lower():
                jfin=jfin+1 
grouped_data = {}

with open(source_text, 'r') as infile:
    j=-1
    for line in infile:
        if line=='\n':
            continue
        
        line=eval(line)
        source=line[0]
        breaker=1
        for i in sourcesofinterest:
            if i.lower() in source.lower():
                breaker=0
        if breaker==1:
            continue
        j=j+1
        
        freq=np.log10(float(line[2]))
        flux=np.log10(float(str(line[3]).split('*')[0]))

        if '-' not in line[5]:
            beam=np.log10(float(line[5]))
        else:
            beams='NA'


        if source not in grouped_data:
            grouped_data[source] = {
                'sfreqs': [], 'sfluxs': [], 'sbeams': [], 'sdates': [],
                'sfreqs2': [], 'sfluxs2': [], 'sbeams2': [], 'sdates2': [],
            }

        if j==0:
            minfreq=freq
            maxfreq=freq
            minflux=flux
            maxflux=flux
            minbeam=beam
            maxbeam=beam
        if j!=0:
            if freq>maxfreq:
                maxfreq=freq
            if freq<minfreq:
                minfreq=freq
            if flux>maxflux:
                maxflux=flux
            if flux<minflux:
                minflux=flux
            if beam>maxbeam:
                maxbeam=beam
            if beam<minbeam:
                minbeam=beam


minfreqp=round_down_to_half_integer(minfreq)
maxfreqp=round_up_to_half_integer(maxfreq)

minfluxp=round_down_to_half_integer(minflux)
maxfluxp=round_up_to_half_integer(maxflux)

minbeamp=round_down_to_half_integer(minbeam)
maxbeamp=round_up_to_half_integer(maxbeam)

interval = maxfreqp - minfreqp
ten_percent = interval * 0.05
minfreq = minfreqp - ten_percent
maxfreq = maxfreqp + ten_percent

interval = maxfluxp - minfluxp
ten_percent = interval * 0.05
minflux = minfluxp - ten_percent
maxflux = maxfluxp + ten_percent

interval = maxbeamp - minbeamp
ten_percent = interval * 0.05
minbeam = minbeamp - ten_percent
maxbeam = maxbeamp + ten_percent


x_tick_positions=np.arange(minfreqp, maxfreqp + 0.5 , 0.5)  # 6 evenly spaced ticks
x_tick_labels = [f"{pos:.2f}" for pos in x_tick_positions]  # Convert log to linear

y_tick_positions = np.arange(minfluxp, maxfluxp + 0.5, 0.5)  # 6 evenly spaced ticks
y_tick_labels = [f"{pos:.2f}" for pos in y_tick_positions]  # Convert log to linear

stick_positions = np.arange(minbeamp, maxbeamp + 1, 1)  # 6 evenly spaced ticks
stick_labels = [f"{pos:.2f}" for pos in stick_positions]  # Convert log to linear

source_text='fitsumfiles/final/fitsummary_final_withextras.txt'

sourcedone=[]
sources=[]

oldsources=[]
oldsource='NA'

with open(source_text, 'r') as infile:
    j=-1
    j2=-1
    for line in infile:
        if len(line)<2:
            continue
        line=eval(line)
        source=line[0]

        breaker=1
        for i in sourcesofinterest:
            if i.lower() in source.lower():
                j2=j2+1 
                breaker=0
        if breaker==1:
            continue
        j=j+1

        if j==0:
            sbeamsizes=[]
            sdates=[]
            sfluxs=[]
            sfreqs=[]
            sbeamsizes2=[]
            sdates2=[]
            sfluxs2=[]
            sfreqs2=[]

            
        breaker=0

        sources.append(source)
        sources=list(set(sources))

        if len(sources)-len(oldsources)!=0 and j!=0:
            source=oldsource

            for isource in sourcedone:
                if source==isource:
                    breaker=1
            if breaker==1:
                continue
            sourcedone.append(source)

            fig, ax = pylab.subplots(dpi=300) 
            sNorm = matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam)
            scalarMap2 = cmx.ScalarMappable(norm=sNorm, cmap=cm2)

            def assign_colors(beamsizes):
                colors = []
                for size in beamsizes:
                    colors.append(scalarMap2.to_rgba(size))  # Use colormap for valid sizes
                return colors
            
            colors2 = assign_colors(sbeamsizes2)
            colors = assign_colors(sbeamsizes)

            if len(sfreqs2)!=0:
                pylab.scatter(sfreqs2,sfluxs2, c=colors2,marker='v',s=msize) 

            if len(sfreqs)!=0:
                pylab.scatter(sfreqs,sfluxs, c=colors,s=msize)

            colorbar1 = fig.colorbar(
                    cmx.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=minbeamp, vmax=maxbeamp), cmap=cm2),
                    label='Beamsize ["]',
                    ax=ax,
                    ticks=stick_positions
                    )
            
            colorbar1.set_label(
                r'log$_{10}$ Beamsize ["]'
            )

            minor_tick_locations = np.arange(minbeamp, maxbeamp, 0.1)
            major_ticks = colorbar1.get_ticks()
            filtered_minor_ticks = [tick for tick in minor_tick_locations if tick not in major_ticks]
            colorbar1.ax.yaxis.set_ticks(filtered_minor_ticks, minor=True)
            
            ax = pylab.gca()
            ax.set_xlim([minfreq, maxfreq])
            ax.set_xticks(x_tick_positions)
            ax.set_xticklabels(x_tick_labels)

            ax.set_ylim([minflux, maxflux]) 
            ax.set_yticks(y_tick_positions)
            ax.set_yticklabels(y_tick_labels)

            minor_locator = MultipleLocator(0.1)
            ax.xaxis.set_minor_locator(minor_locator)
            ax.yaxis.set_minor_locator(minor_locator)
                                
            pylab.yticks(y_tick_positions, y_tick_labels)
            pylab.xlabel(r'log$_{10}$ Frequency [GHz]')
            pylab.title(f'{source} Spectra')
            pylab.ylabel(r'log$_{10}$ Flux [mJy][Beam]$^{-1}$')
            nicename=f'{source} Spectra.pdf'
            if not os.path.exists(f'SpectraGraphs'):
                os.makedirs(f'SpectraGraphs')
            if os.path.exists(f'SpectraGraphs/{nicename}'):
                os.remove(f'SpectraGraphs/{nicename}')
            pylab.savefig(f'SpectraGraphs/{nicename}', dpi=300)
            pylab.show()
            pylab.clf() 

            grouped_data[source]['sfreqs']=sfreqs
            grouped_data[source]['sfluxs']=sfluxs
            grouped_data[source]['sbeams']=sbeamsizes
            grouped_data[source]['sdates']=sdates

            grouped_data[source]['sfreqs2']=sfreqs2
            grouped_data[source]['sfluxs2']=sfluxs2
            grouped_data[source]['sbeams2']=sbeamsizes2
            grouped_data[source]['sdates2']=sdates2


            sbeamsizes=[]
            sfluxs=[]
            sfreqs=[]
            sdates=[]

            sbeamsizes2=[]
            sfluxs2=[]
            sfreqs2=[]
            sdates2=[]

        if j2==jfin-1:
            if line=='\n':
                continue
 
            fig, ax = pylab.subplots(dpi=300)
            sNorm = matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam)
            scalarMap2 = cmx.ScalarMappable(norm=sNorm, cmap=cm2)

            def assign_colors(beamsizes):
                colors = []
                for size in beamsizes:
                    colors.append(scalarMap2.to_rgba(size))  # Use colormap for valid sizes
                return colors
            
            colors2 = assign_colors(sbeamsizes2)
            colors = assign_colors(sbeamsizes)

            if len(sfreqs2)!=0:
                pylab.scatter(sfreqs2,sfluxs2, c=colors2,marker='v',s=msize) 

            if len(sfreqs)!=0:
                pylab.scatter(sfreqs,sfluxs, c=colors,s=msize)

            colorbar1 = fig.colorbar(
                    cmx.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam), cmap=cm2),
                    label='Beamsize ["]',
                    ax=ax,
                    ticks=stick_positions
                    )
            
            colorbar1.set_label(
                r'log$_{10}$ Beamsize ["]'
            )

            minor_tick_locations = np.arange(minbeamp, maxbeamp, 0.1)
            major_ticks = colorbar1.get_ticks()
            filtered_minor_ticks = [tick for tick in minor_tick_locations if tick not in major_ticks]
            colorbar1.ax.yaxis.set_ticks(filtered_minor_ticks, minor=True)
            
            ax = pylab.gca()
            ax.set_xlim([minfreq, maxfreq])
            ax.set_xticks(x_tick_positions)
            ax.set_xticklabels(x_tick_labels)

            ax.set_ylim([minflux, maxflux]) 
            ax.set_yticks(y_tick_positions)
            ax.set_yticklabels(y_tick_labels)

            minor_locator = MultipleLocator(0.1)
            ax.xaxis.set_minor_locator(minor_locator)
            ax.yaxis.set_minor_locator(minor_locator)
                                
            pylab.yticks(y_tick_positions, y_tick_labels)
            pylab.xlabel(r'log$_{10}$ Frequency [GHz]')
            pylab.title(f'{source} Spectra')
            pylab.ylabel(r'log$_{10}$ Flux [mJy][Beam]$^{-1}$')
            nicename=f'{source} Spectra.pdf'
            if not os.path.exists(f'SpectraGraphs'):
                os.makedirs('SpectraGraphs')
            if os.path.exists(f'SpectraGraphs/{nicename}'):
                os.remove(f'SpectraGraphs/{nicename}')
            pylab.savefig(f'SpectraGraphs/{nicename}', dpi=300)
            pylab.show()
            pylab.clf() 

            grouped_data[source]['sfreqs']=sfreqs
            grouped_data[source]['sfluxs']=sfluxs
            grouped_data[source]['sbeams']=sbeamsizes
            grouped_data[source]['sdates']=sdates

            grouped_data[source]['sfreqs2']=sfreqs2
            grouped_data[source]['sfluxs2']=sfluxs2
            grouped_data[source]['sbeams2']=sbeamsizes2
            grouped_data[source]['sdates2']=sdates2


        oldsource=source

        flux_upperb=0
        if len(str(line[3]).split('*'))>1:
            flux_upperb=1

        freq=np.log10(float(line[2]))
        flux=np.log10(float(str(line[3]).split('*')[0]))
        odate=line[4]
        if odate!='NA':
            odate=odate.split('/')
            date=float(odate[0]) + float(odate[1]) / 12 + float(odate[2]) / 365
        else:
            date='NA'

        if '-' not in line[5]:
            beam=np.log10(float(line[5]))
        else:
            beams=line[5].split('-')
            beamavg=(float(beams[0])+float(beams[1]))/2
            beam=np.log10(beamavg)

        if flux_upperb==0:
            sbeamsizes.append(beam)
            sfluxs.append(flux)
            sfreqs.append(freq)
            sdates.append(date)

        if flux_upperb==1:
            sbeamsizes2.append(beam)
            sfluxs2.append(flux)
            sfreqs2.append(freq)
            sdates2.append(date)

        oldsources=copy.deepcopy(sources)

In [ ]:
# functino to Group frequencies dynamically
def group_by_frequency(freqs, fluxes, beams, dates, tolerance):
    grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = [], [], [], []
    current_group_freqs, current_group_fluxes, current_group_beams, current_group_dates = [], [], [], []

    sorted_data = sorted(zip(freqs, fluxes, beams, dates), key=lambda x: x[0])
    for freq, flux, beam, date in sorted_data:
        if not current_group_freqs or abs(freq - current_group_freqs[0]) <= tolerance:
            current_group_freqs.append(freq)
            current_group_fluxes.append(flux)
            current_group_beams.append(beam)
            current_group_dates.append(date)
        else:
            grouped_freqs.append(current_group_freqs)
            grouped_fluxes.append(current_group_fluxes)
            grouped_beams.append(current_group_beams)
            grouped_dates.append(current_group_dates)
            current_group_freqs = [freq]
            current_group_fluxes = [flux]
            current_group_beams = [beam]
            current_group_dates = [date]

    if current_group_freqs:
        grouped_freqs.append(current_group_freqs)
        grouped_fluxes.append(current_group_fluxes)
        grouped_beams.append(current_group_beams)
        grouped_dates.append(current_group_dates)

    return grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates


In [ ]:
#Define frequency ranges for VLA and ALMA bands
vla_bands = [
    (0.0,0.054) ,     #for Rounding 
    (0.054, 0.086),  # 4 Band
    (0.230, 0.470),  # P Band
    (1, 2),          # L Band
    (2, 4),          # S Band
    (4, 8),          # C Band
    (8, 12),         # X Band
    (12, 18),        # Ku Band
    (18, 26.5),      # K Band
    (26.5, 40),      # Ka Band
    (40, 50),        # Q Band
]

alma_bands = [
    (31.3, 45),   # Band 1
    (67, 90),     # Band 2
    (84, 116),    # Band 3
    (125, 163),   # Band 4
    (163, 211),   # Band 5
    (211, 275),   # Band 6
    (275, 373),   # Band 7
    (385, 500),   # Band 8
    (602, 720),   # Band 9
    (787, 950),   # Band 10
]

# Combine all boundaries and create continuous intervals
all_boundaries = sorted(set([freq for band in vla_bands + alma_bands for freq in band]))
non_overlapping_intervals = [(all_boundaries[i], all_boundaries[i + 1]) for i in range(len(all_boundaries) - 1)]

# Remove gaps by ensuring no holes
continuous_intervals = []
for i, (start, end) in enumerate(non_overlapping_intervals):
    if i > 0 and start > continuous_intervals[-1][1]:
        # Fill the gap between the previous interval and the current one
        continuous_intervals.append((continuous_intervals[-1][1], start))
    continuous_intervals.append((start, end))

# Function to check if a frequency is in any interval
def is_frequency_in_intervals(frequency, intervals):
    for start, end in intervals:
        if start <= frequency < end:  # Check if frequency is in the interval
            return True, (start, end)
    return False, None

# Print all intervals for verification
print("\nAll continuous intervals:")
for i, interval in enumerate(continuous_intervals):
    print(f"Interval {i + 1}: {interval[0]}–{interval[1]} GHz")

def group_by_band(freqs, fluxes, beams, dates, intervals):
    grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = [], [], [], []
    interval_mapping = {interval: ([], [], [], []) for interval in intervals}

    for freq, flux, beam, date in zip(freqs, fluxes, beams, dates):
        found, assigned_interval = is_frequency_in_intervals(freq, intervals)
        if found:
            interval_mapping[assigned_interval][0].append(freq)
            interval_mapping[assigned_interval][1].append(flux)
            interval_mapping[assigned_interval][2].append(beam)
            interval_mapping[assigned_interval][3].append(date)

    for interval, (freq_list, flux_list, beam_list, date_list) in interval_mapping.items():
        if freq_list:
            grouped_freqs.append(freq_list)
            grouped_fluxes.append(flux_list)
            grouped_beams.append(beam_list)
            grouped_dates.append(date_list)

    return grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates


In [ ]:
#Generate Statistical Spectral Plots
# Define tolerance for grouping frequencies (log space)
tolerance = .05

#define if it should be grouped dynamicall or by band
grouper='byband'
#grouper='bydynam'

# Generate consistent x-axis and y-axis tick positions
x_tick_positions = np.linspace(minfreq, maxfreq, num=6)  # 6 evenly spaced ticks
x_tick_labels = [f"{10**pos:.2f}" for pos in x_tick_positions]  # Convert log to linear

y_tick_positions = np.linspace(minflux, maxflux, num=6)  # 6 evenly spaced ticks
y_tick_labels = [f"{10**pos:.2f}" for pos in y_tick_positions]  # Convert log to linear

# Apply grouping to all sources
for source, data in grouped_data.items():
    if grouper=='bydynam':
        grouped_data[source]['grouped_sfreqs'], grouped_data[source]['grouped_sfluxs'], grouped_data[source]['grouped_sbeams'], grouped_data[source]['grouped_sdates'] = group_by_frequency(
            data['sfreqs'], data['sfluxs'], data['sbeams'],data['sdates'],tolerance
        )
        grouped_data[source]['grouped_sfreqs2'], grouped_data[source]['grouped_sfluxs2'], grouped_data[source]['grouped_sbeams2'], grouped_data[source]['grouped_sdates2'] = group_by_frequency(
            data['sfreqs2'], data['sfluxs2'], data['sbeams2'], data['sdates2'],tolerance
        )

    if grouper=='byband':
        grouped_data[source]['grouped_sfreqs'], grouped_data[source]['grouped_sfluxs'], grouped_data[source]['grouped_sbeams'], grouped_data[source]['grouped_sdates'] = group_by_band(
            data['sfreqs'], data['sfluxs'], data['sbeams'], data['sdates'], continuous_intervals
        )
        
        grouped_data[source]['grouped_sfreqs2'], grouped_data[source]['grouped_sfluxs2'], grouped_data[source]['grouped_sbeams2'], grouped_data[source]['grouped_sdates2'] = group_by_band(
            data['sfreqs2'], data['sfluxs2'], data['sbeams2'], data['sdates2'], continuous_intervals
        )


# Initialize combined arrays
combined_freqs = []  # Combined frequencies
combined_fluxes = []  # Combined fluxes
combined_beams = []  # Combined beam sizes

for source, data in grouped_data.items():
    # Combine all frequency, flux, and beam data for the current source
    combined_freqs = data['sfreqs'] 
    combined_fluxes = data['sfluxs'] 
    combined_beams = data['sbeams'] 
    combined_dates = data['sdates'] 

    combined_freqs_2 = data['sfreqs2']
    combined_fluxes_2 = data['sfluxs2'] 
    combined_beams_2 = data['sbeams2'] 
    combined_dates_2 = data['sdates2'] 

    # Group the combined data dynamically for the current source
    if grouper=='bydynam':
        grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = group_by_frequency(
            combined_freqs, combined_fluxes, combined_beams, combined_dates, tolerance
        )

    if grouper=='byband':
        grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = group_by_band(
            combined_freqs, combined_fluxes, combined_beams, combined_dates, continuous_intervals
        )

    # Iterate over grouped combined data
    for freq_group, flux_group, beam_group in zip(grouped_freqs, grouped_fluxes, grouped_beams):
        group_size = len(flux_group)
        central_freq = np.mean(freq_group)
        group_size = len(flux_group)
        central_freq = np.mean(freq_group)
        if group_size >= 4:
            pylab.boxplot(flux_group, positions=[central_freq],widths=0.1,
            showfliers=False)  # Customize outliers
        elif group_size == 3:
            # Vertical line with caps at min and max
            min_flux, max_flux = min(flux_group), max(flux_group)
            median_flux = np.median(flux_group)
            pylab.errorbar(
                x=[central_freq], 
                y=[median_flux], 
                yerr=[[median_flux - min_flux], [max_flux - median_flux]], 
                capsize=5,  # Length of caps
                color='k'
            )
            # Add a horizontal line at the median_flux
            pylab.hlines(
                y=median_flux,  # Position of the horizontal line
                xmin=central_freq - 0.05,  # Adjust as needed for length
                xmax=central_freq + 0.05, 
                color='r',  # Color of the line
                linewidth=1  # Thickness of the horizontal line
            )
        elif group_size == 2:
            # Vertical line with caps at min and max (no median)
            min_flux, max_flux = min(flux_group), max(flux_group)
            median_flux = (min_flux + max_flux) / 2  # Just for centering the line
            pylab.errorbar(
                x=[central_freq], 
                y=[median_flux], 
                yerr=[[median_flux - min_flux], [max_flux - median_flux]], 
                capsize=5, 
                color='k'
            )

        elif group_size == 1:
            pylab.scatter([central_freq], [flux_group[0]], c='b', marker='o', s=10)

    if combined_fluxes_2:  # Ensure '2' fluxes exist
        if not combined_fluxes or min(combined_fluxes_2) < min(combined_fluxes):
            # If main fluxes are empty OR '2' minimum is lower
            min_flux_2 = min(combined_fluxes_2)
            min_flux_2_index = combined_fluxes_2.index(min_flux_2)
            min_freq_2 = combined_freqs_2[min_flux_2_index]
            # Mark the lowest value with a downward triangle
            pylab.scatter([min_freq_2], [min_flux_2], c='black', marker='v', s=7, label='Min Flux from 2')


    # Set consistent x and y limits for all plots
    pylab.xlim(minfreq - 0.1, maxfreq + 0.1)  # Add slight padding
    pylab.ylim(minflux - 0.1, maxflux + 0.1)  # Add slight padding
    # Set consistent x-axis ticks and labels
    # Set consistent x-axis and y-axis ticks and labels
    pylab.xticks(x_tick_positions, x_tick_labels)
    pylab.yticks(y_tick_positions, y_tick_labels)

    # Add labels and title
    pylab.xlabel('Frequency [GHz]')
    pylab.ylabel(r'Flux [mJy][Beam]$^{-1}$')
    pylab.title(f'{source} Stats')

    nicename=f'{source} statistical spectra.pdf'
    if not os.path.exists(f'SpectraGraphs/{source}'):
        os.makedirs(f'SpectraGraphs/{source}')
    if os.path.exists(f'SpectraGraphs/{source}/{nicename}'):
        os.remove(f'SpectraGraphs/{source}/{nicename}')
    pylab.savefig(f'SpectraGraphs/{source}/{nicename}', dpi=300)
    pylab.show()
    pylab.clf() 

In [ ]:
#make function to get distances

import os
from astroquery.ned import Ned
import ast  # For safely parsing text file data

import os
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np

from astropy.coordinates import SkyCoord
import numpy as np
import astropy.units as u

import re

import requests
import csv
from scipy import constants
c=constants.c

def is_subsequence(small, large):
    it = iter(large)
    return all(char in it for char in small)

def decompose_string(s):
    parts = re.findall(r"[A-Za-z]+|\d+", s)  # Finds all letter and number sequences
    return parts if parts else [s]

# Input file with source names and coordinates
txtnamesandcoords = 'namesandcoords.txt'

# List of desired reference codes
wantrefs = ['2018ApJ...863..', '2023ApJ...948..', '2020ApJ...890..', '2020ApJ...891..']

def getdists(newnames):
    newrefs=[]
    if not os.path.exists('distancetable.csv'):
        url = 'https://ned.ipac.caltech.edu/Archive/Distances/NED30.5.1-D-17.1.2-20200415.csv'
        response = requests.get(url)

        if response.status_code == 200:
            with open('distancetable.csv', "wb") as file:
                file.write(response.content)
        else:
            return(f"Failed to download. Status code: {response.status_code}")
    
    maserdist=[]
    otherdist=[]
    j1=-1
    j2=-1
    with open('distancetable.csv', "r", newline="", encoding="utf-8") as file:
        reader = csv.reader(file)
        for row in reader:
            textsource=row[3]
            if len(textsource)==0:
                continue
            textdistmm=row[4]
            textdistmmerr=row[5]

            textdistMpc=row[6]
            textmethod=row[7]
            refcode=row[8]


            textsource=textsource.replace(' ','')

            findred=0
            for i in newnames:
                checkers=[]
                for ii in i:
                    ii=ii.split('Galaxy')[0]
                    if is_subsequence(ii, textsource):
                        checkers.append(ii)
                if len(checkers)==len(i):
                    if row[10]:
                        findred=1
                        redshift=row[10]
                        Hubble=row[11]
                        newdist=c*float(redshift)/float(Hubble)*c
                    
                    if textdistmmerr:
                        if float(textdistmmerr)>0:
                            if 'maser' in textmethod.lower():
                                j1=j1+1
                                newrefs.append(refcode)
                                if j1==0:
                                        maserdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]
                                        maserdisuncer=textdistmmerr
                                elif textdistmmerr<maserdisuncer:
                                    maserdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]
                            else:
                                j2=j2+1
                                if j2==0:
                                    otherdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]
                                    otherdisuncer=textdistmmerr
                                elif textdistmmerr<otherdisuncer:
                                    otherdist=[textsource,textdistmm,textdistmmerr,textdistMpc,textmethod,refcode]

    return(maserdist,otherdist,refcode,findred,newrefs)

# Function to process each source
def process_source(source_name, wantrefs, ra, dec,threshold, secondarynames):
  
    """Query NED for redshift data and filter based on references and uncertainties."""
    # Skip excluded sources

    # Query the redshift table from NED
    distances_table = Ned.get_table(source_name, table='redshifts')

    # Extract relevant columns
    redshifts = distances_table['Published Redshift']
    uncertainties = distances_table['Published Redshift Uncertainty']
    refcodes = distances_table['Refcode']

    # Combine into tuples
    entries = [
        (redshift, uncertainty, refcode)
        for redshift, uncertainty, refcode in zip(redshifts, uncertainties, refcodes)
    ]

    # Filter for entries with desired references
    desired_entries = [
        entry for entry in entries if any(ref in entry[2] for ref in wantrefs)
    ]

    entries_with_reference = []
    entries_without_reference = []

    for redshift, uncertainty, refcode in desired_entries:
        if any(ref in refcode for ref in wantrefs):  # Check if refcode contains a wanted reference
            entries_with_reference.append((redshift, uncertainty, refcode))
        else:
            entries_without_reference.append((redshift, uncertainty, refcode))


    reduced_entries = {}

    # Step 2a: Prioritize entries from `entries_with_reference`
    if entries_with_reference:
        for redshift, uncertainty, refcode in entries_with_reference:
            if uncertainty > 0:  # Ignore entries with zero uncertainty
                if refcode not in reduced_entries or uncertainty < reduced_entries[refcode][1]:
                    reduced_entries[refcode] = (redshift, uncertainty)

    if len(reduced_entries)==0:
        # Step 2b: If `entries_with_reference` is empty, use `entries_without_reference`
        for redshift, uncertainty, refcode in entries_without_reference:
            if uncertainty > 0:  # Ignore entries with zero uncertainty
                if refcode not in reduced_entries or uncertainty < reduced_entries[refcode][1]:
                    reduced_entries[refcode] = (redshift, uncertainty)



    # Print results based on the filtering
    redshift='NA'
    uncertainty='NA'

    newnames=[]
    newlist=decompose_string(secondarynames[0][0])

    newnames.append(newlist)
    if len(secondarynames[0][1])!=0:
        if secondarynames[0][1][0]!='NA':
            for i in secondarynames[0][1]:
                if not is_subsequence(secondarynames[0][0], i):
                    newlist=decompose_string(i)
                    newnames.append(newlist)

    maserdists, otherdists, inewrefs, findred, newrefs = getdists(newnames)

    if reduced_entries:
        for irefcode, (iredshift, iuncertainty) in reduced_entries.items():
            redshift=iredshift
            ynfindref='y'
            refcode=irefcode
            if iuncertainty>0:
                uncertainty=iuncertainty
            else:
                input(f'No uncertainty for chosen referencecode: {refcode} , {source}')
                uncertainty='NA'
    else:

        # If no desired references, and no maser distances, find valid entries with uncertainties
        valid_entries = [entry for entry in entries if entry[1] > 0]
        if valid_entries:
            # Select the entry with the lowest uncertainty
            lowest_uncertainty_entry = min(valid_entries, key=lambda x: x[1])
            lowest_redshift, lowest_uncertainty, reference_code = lowest_uncertainty_entry
            redshift=lowest_redshift
            uncertainty= lowest_uncertainty
            refcode=reference_code
            ynfindref='n'
        else:
            input(f"No valid redshift entries found for {source_name} (1)\n")
        if len(maserdists)!=0:
            z_observed = redshift
            if z_observed >= threshold:
                if len(maserdists)!=0 and findred==1:
                        input('new maser source found... adjust accordingly (1)')
        
    z_observed = redshift
    z_uncer=uncertainty

    if z_observed <= threshold:
        if len(maserdists)!=0:
            input('new maser source found... adjust accordingly (2)')
        if len(maserdists)==0:
            textdistmm=float(otherdists[1])
            textdistmmerr=float(otherdists[2])
            textdistMpc=float(otherdists[3])

            newdist=10**((textdistmm)/5-5)
            newdist_uncer=np.log(10)/5*newdist*textdistmmerr
            textmethod=otherdists[4]
            refcode=otherdists[5]

            return([newdist,newdist_uncer],textmethod,'NA',refcode,'DISTANCE',newrefs)

    # Calculate heliocentric velocity
    c = 299792.458  # Speed of light in km/s
    v_helio = z_observed * c
    v_helio_uncer = z_uncer * c

    #Hubble Constant
    #Pesce, APJ 891:L1
    H = 73.9
    H_uncer = 3
    dist=v_helio/H
    dist_uncer=np.sqrt((v_helio_uncer/H)**2 + (v_helio/H**2*H_uncer)**2)

    return([dist,dist_uncer],'NA',ynfindref,refcode,'REDSHIFT', newrefs)

In [ ]:
#Path on local computer to SMBH mass
from pathlib import Path
parent_dir = Path.cwd().parent
Greenepath=f'{parent_dir}/CountingBHS_and_Spectra/Greene_params/Greene_params.txt'

In [ ]:
#date format
from datetime import datetime, timedelta

def decimal_year_to_date(decimal_year):
    year = int(decimal_year)  # Extract the year
    fraction = decimal_year - year  # Extract the decimal part

    # Convert the decimal part to days (assuming 365 days in a year)
    days_elapsed = int(round(fraction * 365))  

    # Compute the actual date by adding days to January 1st of that year
    date = datetime(year, 1, 1) + timedelta(days=days_elapsed)

    # Format as YY.MM.DD
    #return f"{date.year }/{date.month:02}/{date.day:02}"
    return f"{date.year }"

In [ ]:
#get SMBH size
from scipy import constants
from astropy import constants as aconstants
from matplotlib.lines import Line2D

pc=aconstants.pc.value
mass_sun=aconstants.M_sun.value
G=constants.gravitational_constant
pi=constants.pi

def getsize(log_BH_mass_solar,log_BH_mass_solar_uncer,distanceMPc,distanceMPc_uncer,arcsectopc,arcsectopc_unc):
    mass=10**float(log_BH_mass_solar)*mass_sun
    mass_unc=mass*np.log(10)*float(log_BH_mass_solar_uncer)
    mass_fracerror=(mass_unc/mass)*100

    distance=distanceMPc*pc*10**6
    distance_uncer=distanceMPc_uncer*pc*10**6
    dist_fracerror=(distance_uncer/distance)*100
    
    if arcsectopc=='NA':
        m_d=mass/distance
        m_d_unc=np.sqrt((m_d/mass*mass_unc)**2+(m_d/distance*distance_uncer)**2)
        m_d_fracerror=(m_d_unc/m_d)*100
        diam_bhs_microarcsec_fracerror = m_d_fracerror

        diam_bhs_rad=2*np.sqrt(27)*G/(c**2)*m_d
        diam_bhs_deg=diam_bhs_rad*360/(2*pi)
        diam_bhs_arcsec=diam_bhs_deg*60*60
        diam_bhs_microarcsec=diam_bhs_arcsec*10**6

        diam_bhs_microarcsec_unc=diam_bhs_microarcsec/(m_d)*m_d_unc


        return(log_BH_mass_solar, log_BH_mass_solar_uncer, mass_fracerror, distanceMPc, distanceMPc_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror)

        
    diam_bhs=2*np.sqrt(27)*G/(c**2)*mass
    diam_bhs_unc = 2*np.sqrt(27)*G/(c**2)*mass_unc
    
    diam_bhspc=diam_bhs/pc
    diam_bhspc_unc=diam_bhs_unc/pc

    diam_bhs_arsec=diam_bhspc/arcsectopc
    diam_bhs_microarcsec=diam_bhs_arsec*10**6

    diam_bhs_arsec_unc=np.sqrt(
                                (diam_bhspc_unc/arcsectopc)**2
                                + ((-1)*diam_bhspc/arcsectopc**2*arcsectopc_unc)**2
                                )
    
    diam_bhs_microarcsec_unc=diam_bhs_arsec_unc*10**6
    diam_bhs_microarcsec_fracerror = (diam_bhs_microarcsec_unc/diam_bhs_microarcsec)*100

    return(mass, mass_unc, mass_fracerror, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror)

sagmass=6.6
sagmass_unc_up=.12
sagmass_unc_down=.07
sagdist=8.15*10**(-3)
sagdist_unc=.15*10**(-3)

distanceMPc=sagdist
distanceMPc_uncer=sagdist_unc

log_BH_mass_solar=sagmass
log_BH_mass_solar_uncerup=sagmass_unc_up
log_BH_mass_solar_uncerdown=sagmass_unc_down

mass, mass_uncup, mass_fracerrorup, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_uncup, diam_bhs_microarcsec_fracerrorup=getsize(sagmass,sagmass_unc_up,sagdist,sagdist_unc,'NA','NA')
mass, mass_uncdown, mass_fracerrordown, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_uncdown, diam_bhs_microarcsec_fracerrordown=getsize(sagmass,sagmass_unc_down,sagdist,sagdist_unc,'NA','NA')

diam_bhs_microarcsec_uncup=format_number(diam_bhs_microarcsec_uncup)
diam_bhs_microarcsec_uncdown=format_number(diam_bhs_microarcsec_uncdown)
diam_bhs_microarcsec_fracerror=(diam_bhs_microarcsec_fracerrorup+diam_bhs_microarcsec_fracerrordown)/2

sourcehan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'SagA*', linestyle='')
sizechan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'µ" of Scharchild Radius: {format_number(diam_bhs_microarcsec)} $^{{+{diam_bhs_microarcsec_uncup}}}_{{-{diam_bhs_microarcsec_uncdown}}}$ ({format_number(diam_bhs_microarcsec_fracerror)}%)', linestyle='')
disthan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'Distance From Earth: {format_number(distanceMPc)} ± {format_number(distanceMPc_uncer)} MPc ({format_number(dist_fracerror)}%)', linestyle='')
log_BH_mass_solar_uncerup=format_number(float(log_BH_mass_solar_uncerup))
log_BH_mass_solar_uncerdown=format_number(float(log_BH_mass_solar_uncerdown))
mass_fracerror=(mass_fracerrorup+mass_fracerrordown)/2
masshan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'SMBH Mass: {format_number(float(log_BH_mass_solar))} $^{{+{log_BH_mass_solar_uncerup}}}_{{-{log_BH_mass_solar_uncerdown}}}$ log M☉ ({format_number(mass_fracerror)}%)', linestyle='')
handles1=[sourcehan,sizechan,masshan,disthan]

fig, ax = pylab.subplots(dpi=300)

m87mass=9.81
m87mass_unc=0.06
m87dist=16.8
m87dist_uncup=.8
m87dist_uncdown=.7

distanceMPc=m87dist
distanceMPc_uncerup=m87dist_uncup
distanceMPc_uncerdown=m87dist_uncdown

log_BH_mass_solar=m87mass
log_BH_mass_solar_uncer=m87mass_unc

mass, mass_unc, mass_fracerror, distance, distance_uncerup, dist_fracerrorup, diam_bhs_microarcsec, diam_bhs_microarcsec_uncup, diam_bhs_microarcsec_fracerrorup=getsize(m87mass,m87mass_unc,m87dist,m87dist_uncup,'NA','NA')
mass, mass_unc, mass_fracerror, distance, distance_uncerdown, dist_fracerrordown, diam_bhs_microarcsec, diam_bhs_microarcsec_uncdown, diam_bhs_microarcsec_fracerrordown=getsize(m87mass,m87mass_unc,m87dist,m87dist_uncdown,'NA','NA')

diam_bhs_microarcsec_uncup=format_number(diam_bhs_microarcsec_uncup)
diam_bhs_microarcsec_uncdown=format_number(diam_bhs_microarcsec_uncdown)
diam_bhs_microarcsec_fracerror=(diam_bhs_microarcsec_fracerrorup+diam_bhs_microarcsec_fracerrordown)/2

sourcehan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'M87*', linestyle='')
sizechan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'µ" of Scharchild Radius: {format_number(diam_bhs_microarcsec)} $^{{+{diam_bhs_microarcsec_uncup}}}_{{-{diam_bhs_microarcsec_uncdown}}}$ ({format_number(diam_bhs_microarcsec_fracerror)}%)', linestyle='')
dist_fracerror=(distanceMPc_uncerup+distanceMPc_uncerdown)/2
distanceMPc_uncerup=format_number(float(distanceMPc_uncerup))
distanceMPc_uncerdown=format_number(float(distanceMPc_uncerdown))
disthan=Line2D([0], [0], color='red', marker='o', markersize=0, label=f'Distance From Earth: {format_number(distanceMPc)} $^{{+{distanceMPc_uncerup}}}_{{-{distanceMPc_uncerdown}}}$ MPc ({format_number(dist_fracerror)}%)', linestyle='')
masshan=Line2D([0], [0], color='red', marker='o', markersize=0, label=rf'SMBH Mass: {format_number(float(log_BH_mass_solar))} ±{log_BH_mass_solar_uncerdown} log M☉ ({format_number(mass_fracerror)}%)', linestyle='')
handles2=[sourcehan,sizechan,masshan,disthan]

legend1=ax.legend(handles=handles2, loc='upper center')   
ax.add_artist(legend1)
ax.legend(handles=handles1, loc='lower center')   
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_xticks([])  # Remove x-axis ticks
ax.set_yticks([])  # Remove y-axis ticks

# Remove tick labels
ax.set_xticklabels([])  # Remove x-axis labels
ax.set_yticklabels([])  # Remove y-axis labels
if not os.path.exists('Extras'):
    os.makedirs('Extras')
pylab.savefig(f'Extras/SMBH of SagA* and M87')
pylab.clf() 


In [ ]:
#Beam Mismatch Graphs

import ast
from matplotlib.lines import Line2D
from scipy import constants
from astropy import constants as aconstants

tolerance = 0.05

sourcesofinterest=('4258','1194','3079','4945','Circinus','1068','2273','4388','2960','3789')
freqrange=[200,400]

def round_down_to_half_integer(num):
    return math.floor(num * 2) / 2

def round_up_to_half_integer(num):
    return math.ceil(num * 2) / 2

pc=aconstants.pc.value
mass_sun=aconstants.M_sun.value
c=constants.c
G=constants.gravitational_constant
pi=constants.pi

threshold=0.0015

txtnamesandcoords='namesandcoords.txt'
path='Beam Mismatch'
if os.path.exists(path):
    shutil.rmtree(path)
os.mkdir(path)

newrefs=[]
for isource, data in grouped_data.items():
    breaker=1
    for i in sourcesofinterest:
        if i.lower() in isource.lower():
            breaker=0
    if breaker==1:
        continue
    # Main loop to read and process sources from the input file
    if os.path.exists(txtnamesandcoords):
        with open(txtnamesandcoords, 'r') as infile:
            for line in infile:

                # Parse the line into a list of name-coordinate pairs
                params = ast.literal_eval(line)
                for inamecoords in params:                
                    source_name = inamecoords[0][0]
                    if not isource.lower() in source_name.lower():
                        continue
                    source=source_name
                    ra = inamecoords[1][1]
                    dec = inamecoords[1][2]  # Galaxy Dec in sexagesimal format
                    findvals=process_source(source_name, wantrefs, ra, dec, threshold, inamecoords)
                    distanceMPc=findvals[0][0]
                    distanceMPc_uncer=findvals[0][1]
                    method=findvals[1]
                    ynfindref=findvals[2]
                    refcode=findvals[3]
                    red_depend=findvals[4]
                    if len(findvals[5])!=0:
                        for i in findvals[5]:
                            newrefs.append(i)
    else:
        print(f"File {txtnamesandcoords} does not exist.")

    arcsectokpc= np.pi / (180 * 3600) * distanceMPc * 1000
    arcsectopc=arcsectokpc*1000

    arcsectokpc_error = np.pi / (180 * 3600) * distanceMPc_uncer * 1000
    arcsectopc_unc=arcsectokpc_error*1000

    arcsectopc_fracerror=(arcsectopc_unc/arcsectopc)*100

    distance='NA'
    with open(Greenepath, 'r') as infile:
        j=-1
        breaker=1
        for line in infile:
            j=j+1
            if j>20:
                continue
            source_name=line.split(' ')[0]
            source_name=source_name.replace("−","-")
            if "WISEA" in isource:
                isource=isource.split("WISEA")[0]
            if isource.lower() in source_name.lower():
                log_BH_mass_solar=line.split(' ')[3]
                log_BH_mass_solar_offset=line.split(' ')[5]
                breaker=0
        if breaker!=1:
            mass, mass_unc, mass_fracerror, distance, distance_uncer, dist_fracerror, diam_bhs_microarcsec, diam_bhs_microarcsec_unc, diam_bhs_microarcsec_fracerror=getsize(log_BH_mass_solar,log_BH_mass_solar_uncer,distanceMPc,distanceMPc_uncer,arcsectopc,arcsectopc_unc)



    # Filter out elements where date == 'NA'
    valid_sfreqs = [(f, fl, b, d) for f, fl, b, d in zip(data['sfreqs'], data['sfluxs'], data['sbeams'], data['sdates']) if d != 'NA']
    valid_sfreqs2 = [(f, fl, b, d) for f, fl, b, d in zip(data['sfreqs2'], data['sfluxs2'], data['sbeams2'], data['sdates2']) if d != 'NA']

    nvalid_sfreqs = [(f, fl, b, d) for f, fl, b, d in zip(data['sfreqs'], data['sfluxs'], data['sbeams'], data['sdates']) if d == 'NA']
    nvalid_sfreqs2 = [(f, fl, b, d) for f, fl, b, d in zip(data['sfreqs2'], data['sfluxs2'], data['sbeams2'], data['sdates2']) if d == 'NA']

    # Combine all valid data for grouping
    combined_data = valid_sfreqs + valid_sfreqs2
    ncombined_data = nvalid_sfreqs + nvalid_sfreqs2 
    
    # Unpack valid data
    if combined_data:
        combined_freqs, combined_fluxes, combined_beams, combined_dates = zip(*combined_data)
    else:
        combined_freqs, combined_fluxes, combined_beams, combined_dates = [], [], [], []

    if ncombined_data:
        ncombined_freqs, ncombined_fluxes, ncombined_beams, ncombined_dates = zip(*ncombined_data)
    else:
        ncombined_freqs, ncombined_fluxes, ncombined_beams, ncombined_dates = [], [], [], []

    # Group the combined data dynamically for the current source
    grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates = group_by_frequency(
        combined_freqs, combined_fluxes, combined_beams, combined_dates, tolerance
    )

    # Group the combined data dynamically for the current source
    ngrouped_freqs, ngrouped_fluxes, ngrouped_beams, ngrouped_dates = group_by_frequency(
        ncombined_freqs, ncombined_fluxes, ncombined_beams, ncombined_dates, tolerance
    )

    # Iterate through frequency groups and create separate plots
    donefreq=[]
    for freq_group, flux_group, beam_group, date_group in zip(grouped_freqs, grouped_fluxes, grouped_beams, grouped_dates):
        if 10**np.min(freq_group)>freqrange[1]:
            continue
        if 10**np.max(freq_group)<freqrange[0]:
            continue
        donefreq.append(freq_group)
        if len(date_group) > 0 and len(flux_group) > 0:  # Only plot if there are valid points
            totbeams=[]
            totfluxs=[]
            totfreqs=[]
            totdates=[]           # Separate the points by their original data group for plotting
            fig, ax = pylab.subplots(dpi=300)
            #to get maximum values
            for valid_data in [valid_sfreqs, valid_sfreqs2 ]:
                filtered_points = [(d, fl, b) for f, fl, b, d in valid_data if f in freq_group]
                if filtered_points:
                    filtered_dates, filtered_fluxes, filtered_beams = zip(*filtered_points)
                    for i in filtered_beams:

                        i = 10** i
                        i = i * (np.pi / (180 * 3600))
                        i = i * distanceMPc * 1000
                        i = np.log10(i)
                        totbeams.append(i)

                    for i in filtered_fluxes:
                        totfluxs.append(i) 

                    for i in filtered_dates:
                        totdates.append(i)

            maxbeamp=np.max(totbeams)
            minbeamp=np.min(totbeams)

            maxfluxp=np.max(totfluxs)
            minfluxp=np.min(totfluxs)

            maxdatep=np.max(totdates)
            mindatep=np.min(totdates)

            maxbeamp=round_up_to_half_integer(maxbeamp)
            minbeamp=round_down_to_half_integer(minbeamp)

            maxfluxp=round_up_to_half_integer(maxfluxp)
            minfluxp=round_down_to_half_integer(minfluxp)

            maxdatep=np.ceil(maxdatep)
            mindatep=np.floor(mindatep)

            #interval = maxdatep - mindatep
            #ten_percent = interval * 0.05
            ten_percent=0
            mindate = mindatep - ten_percent
            maxdate = maxdatep + ten_percent

            interval = maxfluxp - minfluxp
            ten_percent = interval * 0.05
            minflux = minfluxp - ten_percent
            maxflux = maxfluxp + ten_percent

            interval = maxbeamp - minbeamp
            ten_percent = interval * 0.05
            minbeam = minbeamp - ten_percent
            maxbeam = maxbeamp + ten_percent

            x_tick_positions = np.arange(minbeamp, maxbeamp + 0.5 , 0.5)  # 6 evenly spaced ticks
            x_tick_labels =  [f"{pos:.2f}" for pos in x_tick_positions]  # Convert log to linear

            y_tick_positions = np.arange(minfluxp, maxfluxp + 0.5 , 0.5)  # 6 evenly spaced ticks
            y_tick_labels = [f"{pos:.2f}" for pos in y_tick_positions]  # Convert log to linear

            stick_positions = np.arange(mindatep, maxdatep + 1 , 1)  # 6 evenly spaced ticks
            stick_labels = [decimal_year_to_date(pos) for pos in stick_positions]  # Convert log to linear
            print(stick_labels)

            for label, valid_data, marker, cmap, norm in [
                ("Small Beams Type 1", valid_sfreqs, 'o', cm2, matplotlib.colors.Normalize(vmin=mindate, vmax=maxdate)),
                ("Small Beams Type 2", valid_sfreqs2, 'v', cm2, matplotlib.colors.Normalize(vmin=mindate, vmax=maxdate)),
            ]:            
                #to filter through points for graphing
                filtered_points = [(d, fl, b) for f, fl, b, d in valid_data if f in freq_group]
                if filtered_points:
                    filtered_dates, filtered_fluxes, filtered_beams = zip(*filtered_points)
                    filtered_beams = np.array(filtered_beams)
                    filtered_beams = 10**filtered_beams
                    filtered_beams = filtered_beams * (np.pi / (180 * 3600))
                    filtered_beams = filtered_beams * distanceMPc * 1000
                    filtered_beams = np.log10(filtered_beams)
                    filtered_beams = filtered_beams.tolist()

                    #filtered_fluxes=10**np.array(filtered_fluxes) * (np.pi / (180 * 3600)) * distanceMPc * 1000
                    #filtered_fluxes=filtered_fluxes.tolist()

                    scalar_map = cmx.ScalarMappable(norm=norm, cmap=cmap)
                    colors = scalar_map.to_rgba(filtered_dates)
                    ax.scatter(filtered_beams, filtered_fluxes, label=label, marker=marker, c=colors, s=msize)

            stick2=stick_positions

            if len(stick2)>0:
                colorbar2 = fig.colorbar(
                    cmx.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=mindate, vmax=maxdate), cmap=cm2),
                    label='Date',
                    ax=ax,
                    ticks=stick2
                )
                colorbar2.set_label(
                    "Date",  # Label text
                    #labelpad=3,  # Adjust the distance between the label and the colorbar
                    #fontsize=10,  # Font size
                )

                colorbar2.ax.set_yticklabels(stick_labels)

            minor_tick_locations_date = np.arange(mindatep, maxdatep, 1/4)
            major_ticks = colorbar2.get_ticks()
            filtered_minor_ticks = [tick for tick in minor_tick_locations_date if tick not in major_ticks]
            colorbar2.ax.yaxis.set_ticks(filtered_minor_ticks, minor=True)

            ax = pylab.gca()

            ax.set_xlim([minbeam, maxbeam])
            ax.set_xticks(x_tick_positions)
            ax.set_xticklabels(x_tick_labels)

            ax.set_ylim([minflux, maxflux]) 
            ax.set_yticks(y_tick_positions)
            ax.set_yticklabels(y_tick_labels)

            x_minor_locator = MultipleLocator(0.1)
            y_minor_locator = MultipleLocator(0.1)
            ax.yaxis.set_minor_locator(x_minor_locator)
            ax.xaxis.set_minor_locator(y_minor_locator)


            # Configure the plot
            ax.set_xlabel(r"log$_{10}$ Resolution [kPc]")
            ax.set_ylabel(r'log$_{10}$ Flux [mJy][Beam]$^{-1}$')  # Linear y-axis label
            #ax.set_title(f'{source}: {10**np.mean(freq_group):.0f}GHz\n1" on POS: {format_number(arcsectopc)} Pc\nµ" of Scharchild Radius: {format_number(diam_bhs_microarcsec)}')
            if f'{10**np.min(freq_group):.0f}'==f'{10**np.max(freq_group):.0f}':
                ax.set_title(f'{source}: {10**np.min(freq_group):.0f}GHz')
            else:
                ax.set_title(f'{source}: {10**np.min(freq_group):.0f}-{10**np.max(freq_group):.0f}GHz')

            nicename=f'{source} {round(10**np.mean(np.mean(freq_group)))}GHz.pdf'
            if not os.path.exists(f'Beam Mismatch'):
                os.makedirs(f'Beam Mismatch')
            if os.path.exists(f'Beam Mismatch/{nicename}'):
                os.remove(f'Beam Mismatch/{nicename}')


        # For points with no date
        for nfreq_group, nflux_group, nbeam_group, ndate_group in zip(ngrouped_freqs, ngrouped_fluxes, ngrouped_beams, ngrouped_dates):
            if nfreq_group==freq_group:
                if len(ndate_group) > 0 and len(nflux_group) > 0:  # Only plot if there are valid points
                    for valid_data in [nvalid_sfreqs, nvalid_sfreqs2 ]:
                        filtered_points = [(d, fl, b) for f, fl, b, d in valid_data if f in nfreq_group]
                        if filtered_points:
                            filtered_dates, filtered_fluxes, filtered_beams = zip(*filtered_points)
                            filtered_beams = np.array(filtered_beams)
                            filtered_beams = 10**filtered_beams
                            filtered_beams = filtered_beams * (np.pi / (180 * 3600))
                            filtered_beams = filtered_beams * distanceMPc * 1000
                            filtered_beams = np.log10(filtered_beams)
                            filtered_beams = filtered_beams.tolist()
                            ax.scatter(filtered_beams, filtered_fluxes, label=label, marker=marker, s=msize, facecolor='white', edgecolors='black')

                            for i in filtered_beams:
                                i = 10** i
                                i = i * (np.pi / (180 * 3600))
                                i = i * distanceMPc * 1000
                                i = np.log10(i)
                                totbeams.append(i)

                            for i in filtered_fluxes:
                                totfluxs.append(np.log10(i))     

                            for i in filtered_dates:
                                totdates.append(i)

                    nmaxbeamp=np.max(totbeams)
                    nminbeamp=np.min(totbeams)

                    nmaxfluxp=np.max(totfluxs)
                    nminfluxp=np.min(totfluxs)

                    if nmaxbeamp>maxbeamp:
                        maxbeamp=nmaxbeamp
                        maxbeamp=round_up_to_half_integer(maxbeamp)

                    if nminbeamp<minbeamp:
                        minbeamp=nminbeamp
                        minbeamp=round_down_to_half_integer(minbeamp)

                    if nmaxfluxp>maxfluxp:
                        maxfluxp=nmaxfluxp
                        maxfluxp=round_up_to_half_integer(maxfluxp)

                    if nminfluxp<minfluxp:
                        minfluxp=nminfluxp
                        minfluxp=round_down_to_half_integer(minfluxp)

                    if nmaxbeamp>maxbeamp or nminbeamp<minbeamp:
                        interval = maxbeamp - minbeamp
                        ten_percent = interval * 0.05
                        minbeam = minbeamp - ten_percent
                        maxbeam = maxbeamp + ten_percent

                        x_tick_positions = np.arange(minbeam, maxbeam + 0.5 , 0.5)  # 6 evenly spaced ticks
                        x_tick_labels =  [f"{pos:.2f}" for pos in x_tick_positions]  # Convert log to linear

                        ax.set_xlim([minbeam, maxbeam])
                        ax.set_xticks(x_tick_positions)
                        ax.set_xticklabels(x_tick_labels)
                        ax.xaxis.set_minor_locator(minor_locator)

                    if nminfluxp<minfluxp or nmaxfluxp>maxfluxp:

                        interval = maxfluxp - minfluxp
                        ten_percent = interval * 0.05
                        minflux = minfluxp - ten_percent
                        maxflux = maxfluxp + ten_percent

                        y_tick_positions = np.arange(minflux, maxflux + 0.5 , 0.5)  # 6 evenly spaced ticks
                        y_tick_labels = [f"{pos:.2f}" for pos in y_tick_positions]  # Convert log to linear

                        ax.set_ylim([minflux, maxflux]) 
                        ax.set_yticks(y_tick_positions)
                        ax.set_yticklabels(y_tick_labels)
                        ax.yaxis.set_minor_locator(minor_locator)

        pylab.savefig(f'Beam Mismatch/{nicename}', bbox_inches='tight', dpi=300)
        pylab.show()
        pylab.clf() 

    #for if there is only points that have no date
    for nfreq_group, nflux_group, nbeam_group, ndate_group in zip(ngrouped_freqs, ngrouped_fluxes, ngrouped_beams, ngrouped_dates):
        onlynone=1

        if 10**np.min(nfreq_group)>freqrange[1]:
            continue
        if 10**np.max(nfreq_group)<freqrange[0]:
            continue
        for idone in donefreq:
            if idone==nfreq_group:
                onlynone=0
        if onlynone==1:
            fig, ax = pylab.subplots(dpi=300)
            for valid_data, marker in [(nvalid_sfreqs, 'o'), (nvalid_sfreqs2, 'v') ]:
                filtered_points = [(d, fl, b) for f, fl, b, d in valid_data if f in nfreq_group]
                if filtered_points:
                    filtered_dates, filtered_fluxes, filtered_beams = zip(*filtered_points)
                    filtered_beams = np.array(filtered_beams)
                    filtered_beams = 10 ** filtered_beams * (np.pi / (180 * 3600))
                    filtered_beams = filtered_beams * distanceMPc * 1000
                    filtered_beams = np.log10(filtered_beams)
                    filtered_beams = filtered_beams.tolist()

                    maxbeamp=round_up_to_half_integer(filtered_beams[0])
                    minbeamp=round_down_to_half_integer(filtered_beams[0])

                    maxfluxp=round_up_to_half_integer(filtered_fluxes[0])
                    minfluxp=round_down_to_half_integer(filtered_fluxes[0])

                    interval = maxfluxp - minfluxp
                    ten_percent = interval * 0.05
                    minflux = minfluxp - ten_percent
                    maxflux = maxfluxp + ten_percent

                    interval = maxbeamp - minbeamp
                    ten_percent = interval * 0.05
                    minbeam = minbeamp - ten_percent
                    maxbeam = maxbeamp + ten_percent

                    x_tick_positions = np.arange(minbeamp, maxbeamp + 0.5 , 0.5)  # 6 evenly spaced ticks
                    x_tick_labels =  [f"{pos:.2f}" for pos in x_tick_positions]  # Convert log to linear

                    y_tick_positions = np.arange(minfluxp, maxfluxp + 0.5 , 0.5)  # 6 evenly spaced ticks
                    y_tick_labels = [f"{pos:.2f}" for pos in y_tick_positions]  # Convert log to linear

                    ax.set_xlim([minbeam, maxbeam])
                    ax.set_xticks(x_tick_positions)
                    ax.set_xticklabels(x_tick_labels)

                    ax.set_ylim([minflux, maxflux]) 
                    ax.set_yticks(y_tick_positions)
                    ax.set_yticklabels(y_tick_labels)

                    x_minor_locator = MultipleLocator(0.1)
                    y_minor_locator = MultipleLocator(0.1)
                    ax.yaxis.set_minor_locator(x_minor_locator)
                    ax.xaxis.set_minor_locator(y_minor_locator)

                    ax.scatter(filtered_beams, filtered_fluxes, label=label, marker=marker, s=msize, facecolor='white', edgecolors='black')
            
            if f'{10**np.min(nfreq_group):.0f}'==f'{10**np.max(nfreq_group):.0f}':
                ax.set_title(f'{source}: {10**np.min(nfreq_group):.0f}GHz')
            else:
                ax.set_title(f'{source}: {10**np.min(nfreq_group):.0f}-{10**np.max(nfreq_group):.0f}GHz')
            ax.set_xlabel("Resolution [kPc]")
            ax.set_ylabel(r'Flux [mJy][Beam]$^{-1}$')

            nicename=f'{source} {round(10**np.mean(np.mean(nfreq_group)))}GHz.pdf'
            if not os.path.exists(f'Beam Mismatch/'):
                os.makedirs(f'Beam Mismatch/')
            if os.path.exists(f'Beam Mismatch/{nicename}'):
                os.remove(f'Beam Mismatch/{nicename}')

            pylab.savefig(f'Beam Mismatch/{nicename}', bbox_inches='tight', dpi=300)
            pylab.show()
            pylab.clf() 


In [ ]:
#Create Spectral Plots

insetter=True
msize=10

path = f'SpectraGraphs'
if os.path.exists(path):
    shutil.rmtree(path)
os.mkdir(path)

source_text='fitsumfiles/final/fitsummary_final_withextras.txt'
freqs=[]
fluxs=[]
beams=[]

findwithprocessor=0

#redshifts
noprocessor=[
    ['4258' , 0.0306 , 0.0012],
    ['1194' , 0.268 , 0.011],
    ['1068' , 0.0746 , 0.0030],
    ['5765' , 0.546 , 0.022],
    ['2273' , 0.121 , 0.005],
    ['3393' , 0.251 , 0.010],
    ['Circinus' , 0.02040 , 0.00002],
    ['074-064' , 0.454 , 0.018],
    ['6264' , 0.668 , 0.027],
    ['3789' , 0.214 , 0.009],
    ['4388' , 0.166 , 0.007],
    ['4945' , 0.0369, 0.0015],
    ['6093' , 0.709 , 0.029],
    ['3079' , 0.0726 , 0.0029],
    ['2960' , 0.325 , 0.013],
    ['558' , 0.500 , 0.020],
    ['1320' , 0.183 , 0.007],
    ['2560' , 0.192 , 0.008],
    ['6323' , 0.514 , 0.008],
    ['0437' , 0.316 , 0.013],
    ['1029' , 0.595 , 0.024]
]

with open(source_text, 'r') as infile:
    jfin=-1
    for line in infile:
        if len(line)<2:
            continue
        jfin=jfin+1 

grouped_data = {}

with open(source_text, 'r') as infile:
    j=-1
    for line in infile:
        if line=='\n':
            continue
        
        line=eval(line)
        source=line[0]
        breaker=1
        for i in sourcesofinterest:
            if i.lower() in source.lower():
                breaker=0
        if breaker==1:
            continue
        j=j+1
        
        freq=np.log10(float(line[2]))
        flux=np.log10(float(str(line[3]).split('*')[0]))
        if '-' not in line[5]:
            beam=np.log10(float(line[5]))
        else:
            beams=line[5].split('-')
            beamavg=(float(beams[0])+float(beams[1]))/2
            beam=np.log10(beamavg)


        if source not in grouped_data:
            grouped_data[source] = {
                'sfreqs': [], 'sfluxs': [], 'sbeams': [], 'sdates': [],
                'sfreqs2': [], 'sfluxs2': [], 'sbeams2': [], 'sdates2': [],
            }

        if j==0:
            minfreq=freq
            maxfreq=freq
            minflux=flux
            maxflux=flux
            minbeam=beam
            maxbeam=beam
        if j!=0:
            if freq>maxfreq:
                maxfreq=freq
            if freq<minfreq:
                minfreq=freq
            if flux>maxflux:
                maxflux=flux
            if flux<minflux:
                minflux=flux
            if beam>maxbeam:
                maxbeam=beam
            if beam<minbeam:
                minbeam=beam

minfreqp=round_down_to_half_integer(minfreq)
maxfreqp=round_up_to_half_integer(maxfreq)

minfluxp=round_down_to_half_integer(minflux)
maxfluxp=round_up_to_half_integer(maxflux)

minbeamp=round_down_to_half_integer(minbeam)
maxbeamp=round_up_to_half_integer(maxbeam)

interval = maxfreqp - minfreqp
ten_percent = interval * 0.05
minfreq = minfreqp - ten_percent
maxfreq = maxfreqp + ten_percent

interval = maxfluxp - minfluxp
ten_percent = interval * 0.05
minflux = minfluxp - ten_percent
maxflux = maxfluxp + ten_percent

interval = maxbeamp - minbeamp
ten_percent = interval * 0.05
minbeam = minbeamp - ten_percent
maxbeam = maxbeamp + ten_percent

x_tick_positions=np.arange(minfreqp, maxfreqp + 0.5 , 0.5)  # 6 evenly spaced ticks
x_tick_labels = [f"{pos:.2f}" for pos in x_tick_positions]  # Convert log to linear

y_tick_positions = np.arange(minfluxp, maxfluxp + 0.5, 0.5)  # 6 evenly spaced ticks
y_tick_labels = [f"{pos:.2f}" for pos in y_tick_positions]  # Convert log to linear

stick_positions = np.arange(minbeamp, maxbeamp + 1, 1)  # 6 evenly spaced ticks
stick_labels = [f"{pos:.2f}" for pos in stick_positions]  # Convert log to linear

source_text='fitsumfiles/final/fitsummary_final_withextras.txt'

sourcedone=[]
sources=[]

oldsources=[]
oldsource='NA'

with open(source_text, 'r') as infile:
    j=-1
    j2=-1
    for line in infile:
        j2=j2+1 
        if len(line)<2:
            continue
        line=eval(line)

        source=line[0]
        breaker=1

        for i in sourcesofinterest:
            if i.lower() in source.lower() or source.lower() in i:
                breaker=0
        if breaker==1:
            continue
        j=j+1

        if j==0:
            sbeamsizes=[]
            sdates=[]
            sfluxs=[]
            sfreqs=[]
            sbeamsizes2=[]
            sdates2=[]
            sfluxs2=[]
            sfreqs2=[]

            
        breaker=0

        sources.append(source)
        sources=list(set(sources))


        if len(sources)-len(oldsources)!=0 and j!=0:
            source=oldsource

            for isource in sourcedone:
                if source==isource:
                    breaker=1
            if breaker==1:
                continue
            sourcedone.append(source)

            fig, ax = pylab.subplots(dpi=300)
            sNorm = matplotlib.colors.Normalize(vmin=minbeam, vmax=maxbeam)
            scalarMap2 = cmx.ScalarMappable(norm=sNorm, cmap=cm2)

            def assign_colors(beamsizes):
                colors = []
                for size in beamsizes:
                    colors.append(scalarMap2.to_rgba(size))  # Use colormap for valid sizes
                return colors
            
            colors2 = assign_colors(sbeamsizes2)
            colors = assign_colors(sbeamsizes)

            if len(sfreqs2)!=0:
                pylab.scatter(sfreqs2,sfluxs2, c=colors2,marker='v',s=msize) 

            if len(sfreqs)!=0:
                pylab.scatter(sfreqs,sfluxs, c=colors,s=msize)

            pylab.grid(True)

            colorbar1 = fig.colorbar(
                    cmx.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=minbeamp, vmax=maxbeamp), cmap=cm2),
                    label='Beamsize ["]',
                    ax=ax,
                    ticks=stick_positions
                    )
            
            colorbar1.set_label(
                r'log$_{10}$ Beamsize ["]'
            )

            minor_tick_locations = np.arange(minbeamp, maxbeamp, 0.1)
            major_ticks = colorbar1.get_ticks()
            filtered_minor_ticks = [tick for tick in minor_tick_locations if tick not in major_ticks]
            colorbar1.ax.yaxis.set_ticks(filtered_minor_ticks, minor=True)
            
            ax = pylab.gca()
            ax.set_xlim([minfreq, maxfreq])
            ax.set_xticks(x_tick_positions)
            ax.set_xticklabels(x_tick_labels)

            ax.set_ylim([minflux, maxflux]) 
            ax.set_yticks(y_tick_positions)
            ax.set_yticklabels(y_tick_labels)

            minor_locator = MultipleLocator(0.1)
            ax.xaxis.set_minor_locator(minor_locator)
            ax.yaxis.set_minor_locator(minor_locator)

            



            

            if os.path.exists(txtnamesandcoords):
                with open(txtnamesandcoords, 'r') as infile:
                    for line2 in infile:

                        # Parse the line into a list of name-coordinate pairs
                        params = ast.literal_eval(line2)
                        for inamecoords in params:                
                            source_name = inamecoords[0][0]
                            if not source.lower() in source_name.lower():
                                continue
                            ra = inamecoords[1][1]
                            dec = inamecoords[1][2]  # Galaxy Dec in sexagesimal format
                            if findwithprocessor==1:
                                findvals=process_source(source_name, wantrefs, ra, dec, threshold, inamecoords)
                                distanceMPc=findvals[0][0]
                                distanceMPc_uncer=findvals[0][1]

                                arcsectokpc= np.pi / (180 * 3600) * distanceMPc * 1000
                                arcsectopc=arcsectokpc*1000

                                arcsectokpc_error = np.pi / (180 * 3600) * distanceMPc_uncer * 1000
                                arcsectopc_unc=arcsectokpc_error*1000
                            else:
                                finder=0
                                for findsource,dist,distunc in noprocessor:
                                    if findsource.lower() in source.lower():
                                        arcsectokpc=dist
                                        arcsectokpc_error=distunc
                                        finder=1
                                if finder==0:
                                    input(f"ERROR for {isource} not found in noprocessor list")

            else:
                print(f"File {txtnamesandcoords} does not exist.")

            if insetter==True:
                if arcsectokpc_error!='NA': 
                    if 'circinus' in source.lower():
                        pylab.text(maxfreqp, maxfluxp, f'Arcsecond to Kiloparsec Scale Conversion: {arcsectokpc:.5f} ± {arcsectokpc_error:.5f}', style='italic', bbox={'facecolor': 'grey', 'alpha': 0.2}, horizontalalignment='right', verticalalignment='top', fontsize=7.5)
                    else:
                        pylab.text(maxfreqp, maxfluxp, f'Arcsecond to Kiloparsec Scale Conversion: {arcsectokpc} ± {arcsectokpc_error}', style='italic', bbox={'facecolor': 'grey', 'alpha': 0.2}, horizontalalignment='right', verticalalignment='top', fontsize=7.5)
                else:  
                    pylab.text(maxfreqp, maxfluxp, f'Arcsecond to Kiloparsec Scale Conversion: {arcsectokpc}', style='italic', bbox={'facecolor': 'grey', 'alpha': 0.2}, horizontalalignment='right', verticalalignment='top', fontsize=7.5)



            pylab.yticks(y_tick_positions, y_tick_labels)
            pylab.xlabel(r'log$_{10}$ Frequency [GHz]')
            pylab.title(f'{source} Spectra')
            pylab.ylabel(r'log$_{10}$ Flux [mJy][Beam]$^{-1}$')
            nicename=f'{source} Spectra.pdf'
            if not os.path.exists(f'SpectraGraphs'):
                os.makedirs(f'SpectraGraphs')
            if os.path.exists(f'SpectraGraphs/{nicename}'):
                os.remove(f'SpectraGraphs/{nicename}')
            pylab.savefig(f'SpectraGraphs/{nicename}', dpi=300)
            pylab.show()
            pylab.clf() 

            grouped_data[source]['sfreqs']=sfreqs
            grouped_data[source]['sfluxs']=sfluxs
            grouped_data[source]['sbeams']=sbeamsizes
            grouped_data[source]['sdates']=sdates

            grouped_data[source]['sfreqs2']=sfreqs2
            grouped_data[source]['sfluxs2']=sfluxs2
            grouped_data[source]['sbeams2']=sbeamsizes2
            grouped_data[source]['sdates2']=sdates2


            sbeamsizes=[]
            sfluxs=[]
            sfreqs=[]
            sdates=[]

            sbeamsizes2=[]
            sfluxs2=[]
            sfreqs2=[]
            sdates2=[]


        flux_upperb=0
        if len(str(line[3]).split('*'))>1:
            flux_upperb=1

        freq=np.log10(float(line[2]))
        flux=np.log10(float(str(line[3]).split('*')[0]))
        odate=line[4]
        if odate!='NA':
            odate=odate.split('/')
            date=float(odate[0]) + float(odate[1]) / 12 + float(odate[2]) / 365
        else:
            date='NA'

        if '-' not in line[5]:
            beam=np.log10(float(line[5]))
        else:
            beams=line[5].split('-')
            beamavg=(float(beams[0])+float(beams[1]))/2
            beam=np.log10(beamavg)



        if flux_upperb==0:
            sbeamsizes.append(beam)
            sfluxs.append(flux)
            sfreqs.append(freq)
            sdates.append(date)

        if flux_upperb==1:
            sbeamsizes2.append(beam)
            sfluxs2.append(flux)
            sfreqs2.append(freq)
            sdates2.append(date)

        oldsources=copy.deepcopy(sources)
        oldsource=source

